In [1]:
# Import necessary packages
import pandas as pd
from dash import Dash, html, dash_table, dcc, callback, Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Load the CSV file 
df_original = pd.read_csv("gyro_data.csv")
df_original

,Gyro_X,Gyro_Y,Gyro_Z,Timestamp
0,1.77,-0.37,0.55,2024-08-13 16:04:34
1,0.49,0.92,0.55,2024-08-13 16:04:35
2,0.24,0.55,0.55,2024-08-13 16:04:36
3,0.24,0.43,0.55,2024-08-13 16:04:37
4,0.12,0.37,0.55,2024-08-13 16:04:38
...,...,...,...,...
2820,0.31,0.37,0.61,2024-08-13 16:51:40
2821,0.37,0.37,0.55,2024-08-13 16:51:41
2822,-0.18,0.92,0.55,2024-08-13 16:51:42
2823,0.43,0.67,0.55,2024-08-13 16:51:43


In [3]:
# Initialize the Dash app
app = Dash()

# App layout
app.layout = [
    html.Div(children = "Task 6.1P - Plotly data dashboard") , 
    html.Hr(),
    
    html.Div(children = "Select Types of Displayed Graph"),
    # Dropdown menu for selecting different typs of graph
    dcc.Dropdown(
        id = "graph-type-dropdown", 
        options = [
            {"label" : "Line Plot" ,"value" :"line"},
            {"label" : "Area Plot" ,"value" :"area"},
            {"label" : "Histogram Plot" ,"value" :"histogram"},
            {"label" : "Scatter Plot" ,"value" :"scatter"},
                ],
        value ="line"
    ),
    
    # Dropdown menu for selecting variables
    html.Div(children = "Select Variables to Display"),
    dcc.Dropdown(
        id = "variables-dropdown",
        options = [
            {"label": "X", "value": "Gyro_X"},
            {"label": "Y", "value": "Gyro_Y"},
            {"label": "Z", "value": "Gyro_Z"},
            {"label": "All variables", "value": "All"}
        ],
        value = "Gyro_X"
    ),
    # Number of rows to display in the table
    html.Div(children = "Number of Rows to Display"),
    dcc.Input(id = "num-rows-input" , type ="number" , value = 6 , min =1),
    dash_table.DataTable(id = "table" , data = df_original.to_dict("records"), page_size = 5 ),# Default number of rows
    dcc.Graph(figure ={}, id="dropdown-and-graph" ),
    
    # Summary statistic
    dash_table.DataTable( id = "summary-table",
       columns=[
            {"name": "Variable", "id": "variable"},
            {"name": "Mean", "id": "mean"},
            {"name": "Median", "id": "median"},
            {"name": "Min", "id": "min"},
            {"name": "Max", "id": "max"},
            {"name": "Std Dev", "id": "std_dev"}
        ],
    )
]
@callback(
    
    Output(component_id = "dropdown-and-graph", component_property = "figure"),
    Output(component_id = "table" , component_property = "data"),
    Output(component_id = "summary-table", component_property = "data"),
    # Types of graphf
    Input(component_id = "graph-type-dropdown", component_property ="value"),
    
    #Variables selection
    Input(component_id = "variables-dropdown" , component_property ="value"),
    
    # Number of samples displayed
    Input(component_id = "num-rows-input" , component_property = "value")
)
def update_graph(graph_type, data_vars , num_rows):
    # Validate number of rows
    try:
        if num_rows == None or num_rows <= 0:
            num_rows = 0
    except ValueError:
        num_rows = 5

    # Update the DataTable
    table_data = df_original.head(num_rows).to_dict('records')
    
    df = df_original.head(num_rows)
    # Filter out the dataframe based on the selected variables
    if data_vars == "All":
        filtered_df = df
        y_vars = ["Gyro_X", "Gyro_Y", "Gyro_Z"]
    else:
        filtered_df = df[[data_vars, "Timestamp"]]
        y_vars = [data_vars]

    # Create the appropriate graph based on the selected graph type
    if graph_type == "line":
        fig = px.line(filtered_df, x="Timestamp", y=y_vars, title="Line Plot")
    elif graph_type == "area":
        fig = px.area(filtered_df, x="Timestamp", y=y_vars, title="Area Plot")
    elif graph_type == "histogram":
            fig = px.histogram(filtered_df, x= y_vars, title=f"Histogram of {data_vars}")
    elif graph_type == "scatter":
        if data_vars == "All":
            fig = make_subplots(rows=2, cols=2, subplot_titles=('X vs Y', 'X vs Z', 'Y vs Z'))
            
            fig.add_trace(
                go.Scatter(x=df["Gyro_X"], y=df["Gyro_Y"], mode='markers', name='X vs Y'),
                row=1, col=1
            )
            
            fig.add_trace(
                go.Scatter(x=df["Gyro_X"], y=df["Gyro_Z"], mode='markers', name='X vs Z'),
                row=1, col=2
            )
            
            fig.add_trace(
                go.Scatter(x=df["Gyro_Y"], y=df["Gyro_Z"], mode='markers', name='Y vs Z'),
                row=2, col=1
            )
            
            fig.update_layout(title="Scatter Plots for All Variables")
        else :
            x_var = data_vars
            y_vars = [var for var in ["Gyro_X", "Gyro_Y", "Gyro_Z"] if var != x_var]
            fig = make_subplots(rows=1, cols=len(y_vars), subplot_titles=[f'{x_var} vs {y}' for y in y_vars])
            
            for i, y_var in enumerate(y_vars, start=1):
                fig.add_trace(
                    go.Scatter(x=df[x_var], y=df[y_var], mode='markers', name=f'{x_var} vs {y_var}'),
                    row=1, col=i
             )    
            fig.update_layout(title=f"Scatter Plots for {x_var} with Other Variables")
    # Compute summary statistics for the selected variable(s)
    summary_data = []
    
    for var in y_vars:
        summary_data.append({
            "variable": var,
            "mean": round(filtered_df[var].mean(), 2),
            "median": round(filtered_df[var].median(), 2),
            "min": round(filtered_df[var].min(), 2),
            "max": round(filtered_df[var].max(), 2),
            "std_dev": round(filtered_df[var].std(), 2)
        })
    

    return fig,table_data , pd.DataFrame(summary_data).to_dict("records")
if __name__ == '__main__':
    app.run(debug=True,jupyter_mode = "tab" , port = 8051)

Dash app running on http://127.0.0.1:8051/


<IPython.core.display.Javascript object>